In [28]:
import numpy as np
import numpy.random as nprd
import pandas as pd
import os
import sklearn
from sklearn import svm, tree
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import torch.utils as utils
import torch.utils.data
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import matplotlib.pyplot as plt

import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
device = torch.device('cuda')

import math
import time
import copy
import random as rd

In [2]:
data = pd.read_csv("./ionosphere.data", header=None)

In [8]:
nprd.seed(2020)
data = pd.read_csv("./ionosphere.data", header=None)
data = data.loc[nprd.permutation(len(data)),:]
num_data = len(data)
split_size = num_data // 10
data_split = []
for split_ind in range(9):
    data_split.append(data[split_ind * split_size:(split_ind+1) * split_size])
data_split.append(data[(split_ind+1) * split_size:])

cv_data = []
test_labels = data.loc[:, data.columns[-1]]
for split_ind in range(10):
    split_train = pd.DataFrame()
    for ind in range(10):
        if ind != split_ind:
            split_train = split_train.append(data_split[ind], ignore_index=True)
    cv_data.append([split_train, data_split[split_ind]])
    
class_prop = {'g':sum(data.loc[:,data.columns[-1]] == 'g'),
              'b':sum(data.loc[:,data.columns[-1]] == 'b')}

In [9]:
def test(model):
    for ind in range(10):
        train_data, test_data = cv_data[ind]
        train_X = train_data.loc[:, data.columns[:-1]]
        test_X = test_data.loc[:, data.columns[:-1]]
        train_X = StandardScaler().fit(train_X).transform(train_X)
        test_X = StandardScaler().fit(test_X).transform(test_X)
        train_Y = train_data.loc[:, data.columns[-1]]
        model.fit(train_X, train_Y)

        if ind:
            predicts = np.concatenate((predicts, model.predict(test_X)), axis = 0)
        else:
            predicts = model.predict(test_X)

    print(accuracy_score(test_labels, predicts))
    print(precision_recall_fscore_support(test_labels, predicts, average='binary', pos_label='g'))

In [10]:
test(svm.SVC(C=0.05, kernel='rbf', class_weight=class_prop))

0.9344729344729344
(0.9243697478991597, 0.9777777777777777, 0.9503239740820735, None)


In [11]:
test(tree.DecisionTreeClassifier(criterion = 'entropy', class_weight=class_prop))

0.8660968660968661
(0.8677685950413223, 0.9333333333333333, 0.899357601713062, None)


In [233]:
test(RandomForestClassifier(class_weight=class_prop))

0.9116809116809117
(0.9181034482758621, 0.9466666666666667, 0.9321663019693653, None)


In [206]:
test(AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=1), n_estimators = 100))

0.8774928774928775
(0.8760330578512396, 0.9422222222222222, 0.9079229122055673, None)


In [229]:
class Linear_flipout(nn.Module):
    def __init__(self, in_features, out_features, bias=True, device = "cuda"):
        super(Linear_flipout, self).__init__()
        
        self.device = device
        
        self.in_features = in_features
        self.out_features = out_features
        
        self.weight_mu = nn.Parameter(torch.empty([self.in_features, self.out_features], device = device).double().normal_(0, 1/self.out_features))
        self.weight_logvar = nn.Parameter(torch.ones([self.in_features, self.out_features], device = device).double() *
                                          (- torch.empty(1, device = device).fill_(self.out_features).log()))
        
        self.bias_mu = nn.Parameter(torch.zeros(self.out_features, device = device).double())
        self.bias_logvar = nn.Parameter(torch.ones(self.out_features, device = device).double() *
                                            (- torch.empty(1, device = device).fill_(self.out_features).log()))

            
    def forward(self, x):
        weight_noise = torch.empty(self.weight_mu.shape, requires_grad = False, device = device).normal_(0,1)
        bias_noise = torch.empty(self.bias_mu.shape, requires_grad = False, device = device).normal_(0,1)
        in_sign = torch.empty(x.shape, requires_grad = False, device = device).uniform_(-1,1).sign()
        out_sign = torch.empty([x.shape[0], self.out_features], requires_grad = False, device = device).uniform_(-1,1).sign()

        x = torch.mm(x, self.weight_mu) + torch.mm(in_sign * x, weight_noise * self.weight_mu * self.weight_logvar.div(2).exp()) * out_sign
        x += (1 + bias_noise * self.bias_logvar.div(2).exp()) * self.bias_mu

        kld = (self.weight_mu.pow(2) - self.weight_logvar + self.weight_logvar.exp() - 1).mean()/2
        kld += (self.bias_mu.pow(2) - self.bias_logvar + self.bias_logvar.exp() - 1).mean()/2
        
        return x, kld
    
class bnn(nn.Module):
    def __init__(self):
        super(bnn, self).__init__()
        self.l1 = Linear_flipout(34, 200)
        self.a1 = nn.ELU()
        
        self.l2 = Linear_flipout(200, 200)
        self.a2 = nn.ELU()
        
        self.l3 = Linear_flipout(200, 2)
        self.a3 = nn.ELU()
        
    def forward(self, x):
        x = torch.from_numpy(x).cuda().double()
        
        x, kld1 = self.l1(x)
        x = self.a1(x)
        
        x, kld2 = self.l2(x)
        x = self.a2(x)
        
        x, kld3 = self.l3(x)
        x = self.a3(x)
        
        return x, kld1 + kld2 + kld3
    
    def fit(self, x, y):
        loss = LabelSmoothingLoss(classes = 2, smoothing = 0.1)
        optimizer = optim.SGD(self.parameters(), lr = 1e-3, momentum= 0.9)
        for epoch in range(200):
            optimizer.zero_grad()
            pred, kld = self.forward(x)
            loss(pred, torch.from_numpy((y == 'g').values).long().cuda()).backward()
            optimizer.step()
        optimizer = optim.SGD(self.parameters(), lr = 1e-4, momentum = 0.9)
        for epoch in range(200):
            optimizer.zero_grad()
            pred, kld = self.forward(x)
            loss(pred, torch.from_numpy((y == 'g').values).long().cuda()).backward()
            optimizer.step()
        optimizer = optim.SGD(self.parameters(), lr = 1e-5, momentum = 0.9)
        for epoch in range(200):
            optimizer.zero_grad()
            pred, kld = self.forward(x)
            loss(pred, torch.from_numpy((y == 'g').values).long().cuda()).backward()
            optimizer.step()
            
    def predict(self, x):
        with torch.no_grad():
            pred = self.forward(x)[0]
            res = pd.Series('b').repeat(pred.shape[0])
            res.loc[pred[:,1].cpu().numpy() >= class_prop['g']/(class_prop['g']+class_prop['b'])] = 'g'            
            return res

class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        true_dist = torch.zeros_like(pred)
        true_dist.fill_(self.smoothing / (self.cls - 1))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [230]:
test(bnn().cuda())

0.8888888888888888
(0.9043478260869565, 0.9244444444444444, 0.9142857142857144, None)
